# YOUR PROJECT TITLE

Vi skal: 

1. ændre percentiler til niveauer, og lav standardisering

2. forklar BNP og ulighed på baggrund af governance. 
    - lag governance evt. 10 år.

3. Find varians i BNP og forklar på baggrund af (lagged værdi af) governance

4. Figurer... evt: 
    - Scatterplot over vækst ift. ulighed. 
    - andet?

Imports and set magics:

In [180]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pandas_datareader import wb

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

In [181]:
wb_gini = wb.download(country='all', indicator='SI.POV.GINI', start=1990, end=2017)
wb_gini = wb_gini.rename(columns = {'SI.POV.GINI':'GINI'})
wb_gini = wb_gini.reset_index()
wb_gini.year = wb_gini.year.astype(int) # convert year
wb_gini.country = wb_gini.country.astype('string') # convert country to the special pandas string type
wb_gini.info()

wb_gdp = wb.download(country='all', indicator='NY.GDP.PCAP.KD', start=1990, end=2017)
wb_gdp = wb_gdp.rename(columns = {'NY.GDP.PCAP.KD':'GDP'})
wb_gdp.reset_index(inplace = True)
wb_gdp.year = wb_gdp.year.astype(int)
wb_gdp.country = wb_gdp.country.astype('string') # convert country to the special pandas string type
wb_gdp.info()
wb_gdp.head()

wb = pd.merge(wb_gini, wb_gdp, how = 'outer', on = ['country','year']);
wb.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7448 entries, 0 to 7447
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  7448 non-null   string 
 1   year     7448 non-null   int32  
 2   GINI     1582 non-null   float64
dtypes: float64(1), int32(1), string(1)
memory usage: 145.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7448 entries, 0 to 7447
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  7448 non-null   string 
 1   year     7448 non-null   int32  
 2   GDP      6772 non-null   float64
dtypes: float64(1), int32(1), string(1)
memory usage: 145.6 KB


,country,year,GINI,GDP
0,Africa Eastern and Southern,2017,NaN,1534.683482
1,Africa Eastern and Southern,2016,NaN,1534.924746
2,Africa Eastern and Southern,2015,NaN,1538.552268
3,Africa Eastern and Southern,2014,NaN,1537.159571
4,Africa Eastern and Southern,2013,NaN,1518.482850


In [182]:
wb_wgi=pd.read_csv("WB WGI.csv")

# These columns have to go: 'Unnamed: 0' 'Unnamed: 1' 'Unnamed: 2' 'Unnamed: 3'
drop_these = ['Country Code','Series Code'] # use list comprehension to create list of columns
wb_wgi.drop(drop_these, axis=1, inplace=True) # axis = 1 -> columns, inplace=True -> changed, no copy made
wb_wgi.head(5)

col_dict = {}
for i in range(1996, 2022): # range goes from 1996 to 2021
    col_dict[str(i)+" [YR"+str(i)+"]"] = f'wgi{i}' 
col_dict

wb_wgi.rename(columns = col_dict, inplace=True)
wb_wgi.rename (columns ={'Country Name': 'country'}, inplace=True)
wb_wgi.sample(10)

,country,Series Name,wgi1996,wgi1998,wgi2000,wgi2002,wgi2003,wgi2004,wgi2005,wgi2006,...,wgi2012,wgi2013,wgi2014,wgi2015,wgi2016,wgi2017,wgi2018,wgi2019,wgi2020,wgi2021
581,Jordan,Voice and Accountability: Percentile Rank,42,36.3184089660645,41.7910461425781,25.3731346130371,29.8507461547852,31.730770111084,32.211540222168,27.884614944458,...,26.7605628967285,24.8826293945313,26.6009845733643,25.6157627105713,27.0935955047607,26.6009845733643,27.5362319946289,28.019323348999,28.019323348999,26.0869560241699
303,Cyprus,Regulatory Quality: Percentile Rank,86.9565200805664,85.8695678710938,83.1521759033203,84.3243255615234,85.405403137207,86.0696487426758,84.3137283325195,87.2549057006836,...,83.8862533569336,78.1990509033203,82.6923065185547,80.7692337036133,82.211540222168,81.25,78.3653869628906,80.288459777832,81.25,76.9230804443359
299,Cuba,Voice and Accountability: Percentile Rank,10.5,5.97014904022217,4.97512435913086,4.47761201858521,1.99004971981049,2.88461542129517,2.88461542129517,2.88461542129517,...,6.57277011871338,6.57277011871338,6.40394067764282,6.40394067764282,6.89655160903931,7.38916254043579,9.66183567047119,8.69565200805664,10.1449279785156,11.1111106872559
444,Greenland,Control of Corruption: Percentile Rank,..,..,..,..,..,..,..,..,...,84.3601913452148,83.8862533569336,84.1346130371094,84.1346130371094,83.6538467407227,84.1346130371094,84.1346130371094,83.6538467407227,82.211540222168,82.6923065185547
275,Cook Islands,Voice and Accountability: Percentile Rank,..,..,..,..,..,..,..,..,...,32.8638496398926,32.3943672180176,..,..,..,..,88.8888854980469,81.6425094604492,76.8115921020508,80.6763305664063
1137,Tanzania,Regulatory Quality: Percentile Rank,31.5217399597168,34.7826080322266,33.152172088623,28.1081085205078,29.1891899108887,32.8358192443848,35.7843132019043,38.7254905700684,...,36.9668235778809,39.8104248046875,38.9423065185547,37.980770111084,34.1346168518066,30.2884616851807,29.3269233703613,28.365385055542,27.4038467407227,28.8461532592773
462,Guatemala,Control of Corruption: Percentile Rank,23.6559143066406,26.2032089233398,25.5319156646729,33.3333320617676,27.5132274627686,30.0492610931396,29.2682933807373,21.463415145874,...,28.9099521636963,31.7535552978516,25.9615383148193,24.519229888916,23.5576915740967,23.0769233703613,23.0769233703613,18.75,12.9807691574097,12.0192308425903
281,Costa Rica,Voice and Accountability: Percentile Rank,81.5,84.5771179199219,82.5870666503906,83.5820922851563,76.6169128417969,80.7692337036133,72.5961532592773,75.4807662963867,...,83.5680770874023,83.0985946655273,85.2216720581055,86.2068939208984,84.7290649414063,84.7290649414063,83.5748825073242,83.5748825073242,85.5072479248047,84.5410614013672
27,Andorra,Regulatory Quality: Percentile Rank,88.0434799194336,89.1304321289063,87.5,90.8108139038086,90.2702713012695,89.0547256469727,90.6862716674805,91.1764678955078,...,93.8388595581055,92.890998840332,78.3653869628906,78.8461532592773,78.3653869628906,85.5769195556641,85.5769195556641,86.0576934814453,88.461540222168,88.9423065185547
1212,United Kingdom,Control of Corruption: Percentile Rank,95.1612930297852,96.2566833496094,95.7446823120117,94.7089920043945,95.2380981445313,94.5812835693359,94.634147644043,92.6829299926758,...,92.890998840332,93.3649291992188,93.2692337036133,93.75,94.2307662963867,95.1923065185547,94.2307662963867,93.75,94.2307662963867,93.2692337036133


In [183]:
wb_wgi = pd.wide_to_long(wb_wgi, stubnames='wgi', i=['country','Series Name'], j='year')
wb_wgi = wb_wgi.reset_index()
# Rename colum with series names to "ser"
wb_wgi.rename(columns = {'Series Name':'ser'}, inplace=True)

#Get a list of all the series in the data
namelist = wb_wgi.ser.unique()

newnames = {'Control of Corruption: Percentile Rank':'COC',
            'Government Effectiveness: Percentile Rank':'GOV', 
            'Political Stability and Absence of Violence/Terrorism: Percentile Rank':'RSA', 
            'Regulatory Quality: Percentile Rank':'REQ', 
            'Rule of Law: Percentile Rank':'ROL', 
            'Voice and Accountability: Percentile Rank':'VOA'}

# Renames series
for index, name in enumerate(namelist):
    print(index, "Now rename Series", name, "=", newnames[name])
    
    wb_wgi.loc[wb_wgi.ser == name, 'ser'] = newnames[name]

newnamelist = wb_wgi.ser.unique()

wb_wgi.sample(5)

0 Now rename Series Control of Corruption: Percentile Rank = COC
1 Now rename Series Government Effectiveness: Percentile Rank = GOV
2 Now rename Series Political Stability and Absence of Violence/Terrorism: Percentile Rank = RSA
3 Now rename Series Regulatory Quality: Percentile Rank = REQ
4 Now rename Series Rule of Law: Percentile Rank = ROL
5 Now rename Series Voice and Accountability: Percentile Rank = VOA


,country,ser,year,wgi
19877,North Macedonia,COC,2004,34.4827575683594
28267,Uruguay,VOA,1996,72.5
23619,Slovak Republic,COC,2020,66.3461532592773
2171,Bahrain,ROL,2008,65.3846130371094
17964,Montenegro,GOV,1998,..


In [184]:
wb_new=pd.pivot(wb_wgi, index=['country','year'], columns = 'ser', values= 'wgi')
wb_new=wb_new.reset_index()
final = pd.merge(wb, wb_new, on=['year', 'country'], how = 'outer')

In [185]:
col_list = ['COC', 'GOV', 'REQ', 'ROL', 'RSA', 'VOA']
for i in col_list:
    final.loc[final[i]==".."] = np.nan
    final[i]=final[i].astype(float)
final.info()
final

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8494 entries, 0 to 8493
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  8191 non-null   object 
 1   year     8191 non-null   float64
 2   GINI     1577 non-null   float64
 3   GDP      6624 non-null   float64
 4   COC      4619 non-null   float64
 5   GOV      4619 non-null   float64
 6   REQ      4619 non-null   float64
 7   ROL      4619 non-null   float64
 8   RSA      4619 non-null   float64
 9   VOA      4619 non-null   float64
dtypes: float64(9), object(1)
memory usage: 730.0+ KB


,country,year,GINI,GDP,COC,GOV,REQ,ROL,RSA,VOA
0,Africa Eastern and Southern,2017.0,NaN,1534.683482,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,2016.0,NaN,1534.924746,NaN,NaN,NaN,NaN,NaN,NaN
2,Africa Eastern and Southern,2015.0,NaN,1538.552268,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Eastern and Southern,2014.0,NaN,1537.159571,NaN,NaN,NaN,NaN,NaN,NaN
4,Africa Eastern and Southern,2013.0,NaN,1518.482850,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
8489,Zambia,2021.0,NaN,NaN,25.480770,19.230770,32.211540,30.288462,48.113209,35.265701
8490,Zimbabwe,2018.0,NaN,NaN,10.096154,9.134615,5.288462,8.173077,20.283018,16.908213
8491,Zimbabwe,2019.0,NaN,NaN,10.096154,10.096154,6.250000,7.211538,14.622642,16.425121
8492,Zimbabwe,2020.0,NaN,NaN,9.615385,9.134615,7.692307,7.692307,13.207547,17.874395


#given variabel: træk gennemsnittet fra og divider med standardafvigelsen

Import your data, either through an API or manually, and load it. 

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.